# Import packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.interpolate import interp1d
from PIL import Image, ExifTags
# import datetime as dt
import pytz
from datetime import datetime as DT
import pandas as pd
from lxml import etree
from dateutil import parser

### Input variables

In [2]:
# unique to local files:
homedir = r'/Users/emilysturdivant/Desktop/uas_data'
logfile = os.path.join(homedir, 'f8.gpx')
imagefolder = os.path.join(homedir, 'f8')
rename_photos = False

# Mission info - would be nice for logfile and image folder names to correspond to naming convention, as below
survey_id = '2016-010FA'
uas_id = 'u031'
fc_id = 'f04r01'

# Standard:
namespace = {'def': 'http://www.topografix.com/GPX/1/1'}
tfmt_exif = '%Y:%m:%d %H:%M:%S'
# tfmt_gpx = '%Y-%m-%dT%H:%M:%S-04:00' #2017-05-04T14:14:12-04:00
iso_fmt="%Y%m%dT%H%M%SZ"

### Functions

In [3]:
def dt_to_UTCval(dtstr, fmt, local_tz='US/Eastern'):
    time = (pytz.timezone(local_tz).localize(DT.strptime(e.text, tfmt_gpx), is_dst=None)
                                .astimezone(pytz.utc)
                                .timestamp())
    return(time)

def gpx_tag_to_pdseries(tree, namespace, tag):
    elist = tree.xpath('./def:trk//def:trkpt//def:'+tag, namespaces=namespace)
    ser = pd.Series([e.text for e in elist], name=tag)
    return(ser)

### Parse GPX and extract components into dataframe

In [4]:
# Parse GPX and extract components into dataframe
tree = etree.parse(logfile)

# latitude and longitude
elist = tree.xpath('./def:trk//def:trkpt',namespaces=namespace)
gpxdf = pd.DataFrame([e.values() for e in elist], columns=['lat', 'lon'])

# all other tags
taglist = ['ele', 'ele2', 'course', 'roll', 'pitch', 'mode']
for tag in taglist:
    gpxdf = gpxdf.join(gpx_tag_to_pdseries(tree, namespace, tag))

# time
tag = 'time'
elist = tree.xpath('./def:trk//def:trkpt//def:'+tag, namespaces=namespace)
dt = [parser.parse(e.text) for e in elist] # parser will detect time zones
dtz = [dti.astimezone(pytz.utc) for dti in dt]
gpxdf = gpxdf.join(pd.DataFrame({'time_utc':dtz, 'time_epoch': [t.timestamp() for t in dtz]}))

# gpxdf.head()

# Export CSV: logfile_gpx.csv stored in same folder as logfile
gpxdf.to_csv(os.path.splitext(logfile)[0]+'_gpx.csv', index=False)

[datetime.datetime(2017, 6, 13, 10, 38, 22, tzinfo=tzoffset(None, -14400)), datetime.datetime(2017, 6, 13, 10, 38, 22, tzinfo=tzoffset(None, -14400)), datetime.datetime(2017, 6, 13, 10, 38, 22, tzinfo=tzoffset(None, -14400)), datetime.datetime(2017, 6, 13, 10, 38, 23, tzinfo=tzoffset(None, -14400))]
[datetime.datetime(2017, 6, 13, 14, 38, 22, tzinfo=<UTC>), datetime.datetime(2017, 6, 13, 14, 38, 22, tzinfo=<UTC>), datetime.datetime(2017, 6, 13, 14, 38, 22, tzinfo=<UTC>), datetime.datetime(2017, 6, 13, 14, 38, 23, tzinfo=<UTC>)]


#### Export CSV

In [5]:
#%% PLOT!
fig = plt.figure()
ax = fig.add_subplot(111)
ax.axis([-75.7502, -75.7543, 36.18135, 36.1865])

ax.plot(gpxdf.lon,gpxdf.lat,'-')
# plt.show()
fig.savefig(os.path.join(homedir, "plot_gpxtrack.png"))

## Work with images
Replace original images because this will consider current filenames "original" and will replace. 

In [6]:
# List all JPEGS in imagefolder
flist=[os.path.join(imagefolder,f) for f in os.listdir(imagefolder) if f.lower().endswith('.jpg')]
print("Found {} images in {}.".format(len(flist),imagefolder))

# Get filename and DateTimeOriginal of each photo
#FIXME: how to get tzinfo from EXIF? Looks like these were recorded in UTC...
dt = [pytz.utc.localize(DT.strptime(Image.open(f)._getexif()[36867], tfmt_exif)) for f in flist] # make timezone aware (UTC) for later comparison with GPX times
imgdf = pd.DataFrame({'orig_name': [os.path.basename(f) for f in flist],
                      'time_utc': dt,
                      'time_epoch': [t.timestamp() for t in dt],
                      'time_iso': [t.strftime(iso_fmt) for t in dt],
                      'new_name': np.nan,
                      'lon': np.nan,
                      'lat': np.nan,
                      'ele': np.nan,
                      'interpolated': 0})
# imgdf.head()

# Export CSV
imgdf.to_csv(imagefolder+'_EXIFtime.csv', index=False)

# print first and last image name and times
print("First... file: {}, time: {}".format(imgdf.orig_name.iloc[0],imgdf.time_utc.iloc[0]))
print("Last... file: {}, time: {}".format(imgdf.orig_name.iloc[-1],imgdf.time_utc.iloc[-1]))
# print first and last times in .gpx file
print("{} from {} to {}".format(logfile, gpxdf.time_utc.iloc[0],gpxdf.time_utc.iloc[-1]))

Found 188 images in /Users/emilysturdivant/Desktop/uas_data/f8.
First... file: B0009085.JPG, time: 2017-06-13 14:34:54+00:00
Last... file: B0009316.JPG, time: 2017-06-13 14:49:00+00:00
/Users/emilysturdivant/Desktop/uas_data/f8.gpx from 2017-06-13 14:38:22+00:00 to 2017-06-13 14:55:08+00:00


### Rename photos

In [7]:
# Rename photos
#TODO move/copy them first? / don't run if the names have already been changed...
for idx, row in imgdf.iterrows():
    img = row.orig_name
    namestr = "{}_{}_{}_{}_{}".format(survey_id, uas_id, fc_id, row.time_iso, img) # ->
    if rename_photos:
        os.rename(os.path.join(imagefolder, img), os.path.join(imagefolder, namestr))
    imgdf.loc[idx, 'new_name'] = namestr

# Interpolation

In [8]:
# set up interpolation 
# Datetime objects can be compared, but both need to be either tz-aware or unaware
# Datetime objects cannot be used for interpolation with interp1
timecol = 'time_epoch'

# get GPS bounds
start_gpxtime = gpxdf[timecol].min()
end_gpxtime = gpxdf[timecol].max()
print('GPS start: {}\nGPS end: {}'.format(start_gpxtime, end_gpxtime))

# create interpolator
set_interp = interp1d(gpxdf[timecol], gpxdf[['lat','lon','ele']], kind='linear', axis=0) # this works

# loop through the images and interpolate .gpx data
img_nogps = []
for idx, row in imgdf.iterrows():
    img_tznum = row.loc[timecol]
    if img_tznum >= start_gpxtime and img_tznum <= end_gpxtime:
        imgdf.loc[idx, ['lat','lon','ele']] = set_interp(img_tznum)
    else: # image time is not within .gpx data
        imgdf.loc[idx, ['lat','lon','ele']] = np.nan
        img_nogps.append(row.orig_name)
print('{} image(s) not acquired between {} and {}:'.format(len(img_nogps), start_gpxtime, end_gpxtime))
[print(img) for img in img_nogps]

# Set interpolated value where no GPS time at image time
missing_idx = ~(imgdf[timecol].isin(gpxdf[timecol])) # All images with time logged by GPS
imgdf.loc[missing_idx, 'interpolated'] = 1

# Export CSV
imgdf.to_csv(imagefolder+'_EXIFtime.csv', index=False)

GPS start: 1497364702.0
GPS end: 1497365708.0
1 image(s) not acquired between 1497364702.0 and 1497365708.0:
B0009085.JPG


In [9]:
# make a bare-bones trackline and overlay image locations, sherwood direct
#%% Plot image locations
fig = plt.figure()
ax = fig.add_subplot(111)
ax.axis([-75.7502, -75.7543, 36.18135, 36.1865])
ax.plot(gpxdf.lon, gpxdf.lat,'.r')
ax.plot(imgdf.lon, imgdf.lat,'-')
# plt.show()
fig.savefig(os.path.join(homedir, "image_track.png"))

#%% Plot
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(gpxdf.time_utc, gpxdf.ele,'.r')
ax.plot(imgdf.time_utc, imgdf.ele,'-')
# plt.show()
fig.savefig(os.path.join(homedir, "image_elevations.png"))

In [19]:
# print out file name, time, and data
# TODO - write a .csv file with columns in the correct order for Photoscan
# photoscan_cols = ['Label', 'X', 'Y', 'Z']
psdf = imgdf.loc[:,['new_name','lon','lon','ele']]
psdf.rename(columns={'new_name':'Label', 'lon':'X', 'lon':'Y', 'ele':'Z'}, inplace=True)
psdf.to_csv(os.path.join(homedir,'cam_locations.txt'), index=False)

/Users/emilysturdivant/anaconda/envs/IOOS3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [10]:
# # experiments:
# # select images with a matching GPS time and those without
# matches = imgdf[imgdf[timecol].isin(gpxdf[timecol])] # All images with time logged by GPS
# missing_idx = ~(imgdf[timecol].isin(gpxdf[timecol])) # All images with time logged by GPS
# imgdf.loc[missing_idx, 'interpolated'] = 1
# print('Images with exact GPS time match: ', matches.shape[0])
# print('Images without exact GPS time match: ', missing.shape[0])

# # set up interpolation 
# # gpx4interp = gpxdf[['lat','lon','ele']]
# # set_interp = interp1d(gpxdf[timecol], gpx4interp, kind='linear', axis=0)
# # imgdf.apply(set_interp(timecol), axis=1)

# # gpxdf.interpolate(method='time') 
# # # for use filling NaNs: nans in gaps in data, then interpolate, 
# # # 'time' interpolation works on daily and higher resolution data to interpolate given length of interval
# # # axis: # 0: fill column-by-column # 1: fill row-by-row
# # #set time to index, then reindex to consistent time steps, it will add missing times and put nan values; then use apply to interpolate
# timecol = 'time_utc'
# # drop duplicate time values; alternative: add specificity to them... 
# df_deduped = gpxdf.drop_duplicates(subset=timecol, keep='first')
# df = df_deduped.set_index(timecol)
# df_reindexed = df.reindex(pd.date_range(start=df.index.min(),end=df.index.max()), copy=True)  
# df_reindexed.index
# # print(df.index.min(), df.index.max())
# # df_reindexed.head()
# df_reindexed.interpolate(method='linear')  

# # df2 = df.reindex(arange(time))
# # df2.apply(pandas.Series.interpolate)

# # # alternative:
# # def getExtrapolatedInterpolatedValue(x, y):
# #     global dataGrid
# #     if x not in dataGrid.index:
# #         dataGrid.ix[x] = nan
# #         dataGrid = dataGrid.sort()
# #         dataGrid = dataGrid.interpolate(method='index', axis=0).ffill(axis=0).bfill(axis=0)

# #     if y not in dataGrid.columns.values:
# #         dataGrid = dataGrid.reindex(columns=numpy.append(dataGrid.columns.values, y))
# #         dataGrid = dataGrid.sort_index(axis=1)
# #         dataGrid = dataGrid.interpolate(method='index', axis=1).ffill(axis=1).bfill(axis=1)

# #     return dataGrid[y][x]
# # print getExtrapolatedInterpolatedValue(2, 1.4)
# # #
